In [1]:
#This is based on https://github.com/aniruddhraghu/sepsisrl/blob/master/preprocessing/process_interventions.ipynb

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame

In [4]:
orig_data = pd.read_csv("../Matlab/MIMIC_mimiciii.csv")

In [5]:
len(orig_data)

278578

In [6]:
interventions = orig_data[["max_dose_vaso", "input_4hourly"]]

In [7]:
adjusted_vaso = interventions["max_dose_vaso"][interventions["max_dose_vaso"] >0]
adjusted_iv = interventions["input_4hourly"][interventions["input_4hourly"]>0]

In [8]:
vaso_quartiles = adjusted_vaso.quantile([0.25,0.50,0.75])
iv_quartiles = adjusted_iv.quantile([0.25,0.5,0.75])

In [9]:
vq = np.array(vaso_quartiles)
vq

array([0.08, 0.2 , 0.45])

In [10]:
ivq = np.array(iv_quartiles)
ivq

array([ 48.2670625, 150.       , 500.       ])

In [11]:
import copy
discretised_int = copy.deepcopy(interventions)

In [12]:
discretised_int['vaso_input'] = discretised_int['max_dose_vaso']
discretised_int['vaso_input'][interventions['max_dose_vaso'] == 0.0] = 0
discretised_int['vaso_input'][(interventions['max_dose_vaso'] > 0.0) & (interventions['max_dose_vaso'] < vq[0])] = 1
discretised_int['vaso_input'][(interventions['max_dose_vaso'] >= vq[0]) & (interventions['max_dose_vaso'] < vq[1])] = 2
discretised_int['vaso_input'][(interventions['max_dose_vaso'] >= vq[1]) & (interventions['max_dose_vaso'] < vq[2])] = 3
a = interventions['max_dose_vaso'] >= vq[2]
discretised_int['vaso_input'][a] = 4

In [13]:
discretised_int['iv_input'] = discretised_int['input_4hourly']
discretised_int['iv_input'][interventions['input_4hourly'] == 0.0] = 0
discretised_int['iv_input'][(interventions['input_4hourly'] > 0.0) & (interventions['input_4hourly'] < ivq[0])] = 1
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[0]) & (interventions['input_4hourly'] <  ivq[1])] = 2
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[1]) & (interventions['input_4hourly'] < ivq[2])] = 3
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[2])] = 4

In [14]:
disc_inp_data = copy.deepcopy(orig_data)

In [15]:
disc_inp_data['vaso_input'] = discretised_int['vaso_input']
disc_inp_data['iv_input'] = discretised_int['iv_input']

In [16]:
#disc_inp_data.to_csv('../Created Data/discretised_input_data.csv', index=False)

In [17]:
len(disc_inp_data)

278578

In [18]:
disc_inp_data['vaso_input'].value_counts()

0.0    236666
3.0     11820
4.0     10767
1.0     10182
2.0      9143
Name: vaso_input, dtype: int64

In [19]:
disc_inp_data['iv_input'].value_counts()

0.0    77330
3.0    51005
4.0    50345
1.0    50312
2.0    49586
Name: iv_input, dtype: int64

In [20]:
actions = []
for i, row in disc_inp_data.iterrows():
    if row["vaso_input"] == 0:
        if row["iv_input"] == 0:
            actions.append(0)
        elif row["iv_input"] == 1:
            actions.append(5)
        elif row["iv_input"] == 2:
            actions.append(10)
        elif row["iv_input"] == 3:
            actions.append(15)
        else:
            actions.append(20)
    elif row["vaso_input"] == 1:
        if row["iv_input"] == 0:
            actions.append(1)
        elif row["iv_input"] == 1:
            actions.append(6)
        elif row["iv_input"] == 2:
            actions.append(11)
        elif row["iv_input"] == 3:
            actions.append(16)
        else:
            actions.append(21)
    elif row["vaso_input"] == 2:
        if row["iv_input"] == 0:
            actions.append(2)
        elif row["iv_input"] == 1:
            actions.append(7)
        elif row["iv_input"] == 2:
            actions.append(12)
        elif row["iv_input"] == 3:
            actions.append(17)
        else:
            actions.append(22)
    elif row["vaso_input"] == 3:
        if row["iv_input"] == 0:
            actions.append(3)
        elif row["iv_input"] == 1:
            actions.append(8)
        elif row["iv_input"] == 2:
            actions.append(13)
        elif row["iv_input"] == 3:
            actions.append(18)
        else:
            actions.append(23)
    else:
        if row["iv_input"] == 0:
            actions.append(4)
        elif row["iv_input"] == 1:
            actions.append(9)
        elif row["iv_input"] == 2:
            actions.append(14)
        elif row["iv_input"] == 3:
            actions.append(19)
        else:
            actions.append(24)
disc_inp_data["action"] = actions    

In [21]:
disc_inp_data.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,input_total,input_4hourly,output_total,output_4hourly,cumulated_balance,SOFA,SIRS,vaso_input,iv_input,action
0,1,3,7245486000,0,17639.8264,0,0,0.0,NaN,1.0,...,5127.0,50.0,13617.0,520.0,-8490.0,6,2,0.0,2.0,10
1,2,3,7245500400,0,17639.8264,0,0,0.0,NaN,1.0,...,5177.0,50.0,13982.0,365.0,-8805.0,6,2,0.0,2.0,10
2,3,3,7245514800,0,17639.8264,0,0,0.0,NaN,1.0,...,5227.0,50.0,14262.0,280.0,-9035.0,6,2,0.0,2.0,10
3,4,3,7245529200,0,17639.8264,0,0,0.0,NaN,1.0,...,5277.0,50.0,14602.0,340.0,-9325.0,6,1,0.0,2.0,10
4,5,3,7245543600,0,17639.8264,0,0,0.0,NaN,1.0,...,5327.0,50.0,15142.0,540.0,-9815.0,6,2,0.0,2.0,10


In [22]:
disc_inp_data.to_csv('../Fresh_start/discretised_input_data_MIMIC_mimiciii.csv', index=False)